In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [ ]:
df = pd.read_excel('Smoking/smoking_estimates_means.xlsx')
df_ci = pd.read_excel('Smoking/smoking_estimates_ci.xlsx')

In [ ]:
df.head()

In [ ]:
df_both = df[df.Sex=='Both']

In [ ]:
df_both.shape

In [ ]:
df_both.head()

In [ ]:
plt.scatter(df_both.County, df_both[1996])
#df_both[1996]
#df_both.columns

In [ ]:
plt.scatter(df_both[df_both.State=='CO'].County,df_both[df_both.State=='CO'][1996])

In [ ]:
mean_rates = []
for i in range(1996, 2012):
    mean_rates.append(df_both[i].mean())

In [ ]:
plt.scatter(range(1996, 2012), mean_rates)

In [ ]:
max_increase = df_both['Annualized rate of change, 1996-2012'].sort_values().nlargest(10)
max_decrease = df_both['Annualized rate of change, 1996-2012'].sort_values().nsmallest(10)

In [ ]:
max_decrease

In [ ]:
#df_both.loc[8501,[1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012]]
df_both.loc[8501,'County']

In [ ]:
max_increase = df_both[['State & County','Annualized rate of change, 1996-2012']].sort_values(by='Annualized rate of change, 1996-2012').nlargest(10, columns=['Annualized rate of change, 1996-2012'])
max_decrease = df_both[['State & County','Annualized rate of change, 1996-2012']].sort_values(by='Annualized rate of change, 1996-2012').nsmallest(10, columns=['Annualized rate of change, 1996-2012'])

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))

for i in max_increase['State & County']:
    ax.plot(range(1996, 2013),df_both[df_both['State & County']==i].iloc[:,1:18].T)

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))

for i in max_decrease['State & County']:
    ax.plot(range(1996, 2013),df_both[df_both['State & County']==i].iloc[:,1:18].T)

In [ ]:
df_lung = pd.read_csv('/home/davidhenslovitz/Galvanize/ZNAHealth/CSV/RESPIR.csv', converters={'State-county recode': lambda x: str(x), 'CS Schema v0204+': lambda x: str(x)})

In [ ]:
df_lung = df_lung.replace(r'\s+', np.nan, regex=True)

In [ ]:
df_lung = df_lung[df_lung['CS Schema v0204+']=='063']

In [ ]:
df_lung["state"] = df_lung["State-county recode"].str[0:2]

In [ ]:
 the case of the standardization is quite different, we don’t count with boundaries, we know that in theory in many cases, the majority of the new numbers will be located between -3 and 3, but we can get less or more than that.

Well, the formulas are there and opinions are everywhere, so which method should we pick?
I’ll say both. There is a kind of a rule of thumb that says if your data varies a lot choose standardization, df_lung["county"] = df_lung["State-county recode"].str[2:5]

In [ ]:
df_lung.state.value_counts()

In [ ]:
fips = pd.read_excel('FIPS.xlsx')

In [ ]:
fips.FIPS = fips.FIPS.apply(lambda x: str(x).zfill(5))

In [ ]:
fips.head()

In [ ]:
# for i in df_lung["State-county recode"]:
#     state = fips[fips.FIPS==i]["Name"]

In [ ]:
codes = list(df_lung["State-county recode"])
codes

In [ ]:
lst = []
lst2 = []
for i in codes:
    lst.append(fips[fips.FIPS==i]['Name'].values)
    lst2.append(fips[fips.FIPS==i]['State'].values)
# fips[fips.FIPS=='09009']['Name']

    

In [ ]:
df_lung["County"] = lst
df_lung["County"] = df_lung["County"].str.get(0)
df_lung["State"] = lst2
df_lung["State"] = df_lung["State"].str.get(0)

In [ ]:
df_lung.head(10)

In [ ]:
df_lung["State_and_county"] = df_lung["County"]+" County, "+df_lung["State"]

In [ ]:
df_lung.head()

In [ ]:
counter = 0
lst = []
for i in df_lung["State_and_county"]:
    if (pd.isnull(i) or i not in list(df_both['State & County'].values)):
        lst.append(np.nan)
        
    else:
        ind = list(df_both['State & County'].values).index(i)
        lst.append(ind)
#     if counter > 1000:
#         break
    counter += 1
    
# arr = []
# for x in lst:
#     new_list = []
#     for year in range(1996,2013):
#         new_list.append(df_both.iloc[x][1:18][year])
#     arr.append(new_list)



# for yr in df_both.iloc[1][1:19].index:
#     df_lung[str(yr)] = [i[0] for i in arr]

In [ ]:
print(lst[0])

In [ ]:
arr = []
for x in lst:
    new_list = []
    for year in range(1996,2013):
        if np.isnan(x):
            new_list.append(np.nan)
        else:
            new_list.append(df_both.iloc[x][1:18][year])
    arr.append(new_list)

In [ ]:
df_both.iloc[x][1:19][1999]

In [ ]:
df_both.iloc[302][1:19][1996]

In [ ]:
[i[16] for i in arr]

In [ ]:
counter = 0
for yr in df_both.iloc[1][1:18].index:
    df_lung[str(yr)] = [i[counter] for i in arr]
    counter += 1

In [ ]:
df_lung.drop('Annualized rate of change, 1996-2012', axis=1, inplace=True)

In [ ]:
df_lung.head()

In [ ]:
df_first = df_lung[['Race/Ethnicity','Sex','Age at diagnosis', 'Year of Birth','CS Schema v0204+','Race recode (White, Black, Other)','Race recode (W, B, AI, API)','Origin recode NHIA (Hispanic, Non-Hisp)','State-county recode','state','county','County','State','State_and_county','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012']]

In [ ]:
df_first.shape

In [2]:
df_air = pd.read_csv('Air_Quality_Measures_on_the_National_Environmental_Health_Tracking_Network.csv')

In [3]:
df_pm2 = df_air[df_air.MeasureName=='Annual average ambient concentrations of PM 2.5 in micrograms per cubic meter, based on seasonal averages and daily measurement (monitor and modeled data)']

In [4]:
df_pm2['CountyFips'] = df_pm2['CountyFips'].apply(lambda x: str(x).zfill(5))

/home/davidhenslovitz/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
df_pm2 = df_pm2[['CountyFips', 'ReportYear','Value','UnitName']]

In [6]:
df_pm2.head()

,CountyFips,ReportYear,Value,UnitName
184435,01001,2001,13.186807,Micograms per cubic meter
184436,01001,2002,12.490668,Micograms per cubic meter
184437,01001,2003,12.869481,Micograms per cubic meter
184438,01001,2004,13.111945,Micograms per cubic meter
184439,01001,2005,13.543276,Micograms per cubic meter


In [7]:
df_test = df_first.iloc[:100]
df_pm2new = df_pm2.pivot(index='CountyFips', columns='ReportYear', values='Value')
#list(df_pm2new.index).index('01003')

NameError: name 'df_first' is not defined

In [ ]:

lst = []
for i in df_test['State-county recode']:
    if (i not in list(df_pm2new.index)):
        lst.append(np.nan)
        
    else:
        ind = list(df_pm2new.index).index(i)
        lst.append(ind)

In [ ]:
arr = []
for x in lst:
    new_list = []
    for col in df_pm2new.columns:
        if np.isnan(x):
            new_list.append(np.nan)
        else:
            new_list.append(df_pm2new.iloc[x][col])
    arr.append(new_list)

In [ ]:
counter = 0
for col in df_pm2new.columns:
    df_test[str(col)+"air"] = [i[counter] for i in arr]
    counter += 1


In [ ]:
#df_test.drop(['ReportYear', 'Value','UnitName','CountyFips'],axis=1, inplace=True)
df_test.head()

In [ ]:
# df_pm2.groupby('CountyFips')


In [ ]:
#df_pm2new.loc['01003']
df_pm2new.index

In [ ]:
#True in [pd.isnull(i) for i in df_lung['State-county recode']]
df_pm2new.head()